# VIII. Automating your Experiment

In [1]:
from unittest.mock import Mock, patch
import random
from psychopy import core, visual, sound
from psychopy.hardware import keyboard
import pytest
import ipytest
ipytest.autoconfig()

## 1. Mocking and Patching

Mocking and patching are complementary techniques for replacing components of your code with simulations. <br>
A `Mock` is a special object that simulates a specific behavior and provides a predetermined response.<br>
Patching is the mechanism that lets you temporarily replace specific parts of you code with mocks.<br>
Think of mocking as creating a stand-in actor, while patching is placing that actor on stage at the right moment.<br>
For example, if you have code that reads from a file, you can create a mock object that simulates file operations.<br>
Now, you can use patching to temporarily replace Python's built-in `open` function with your mock. <br>
This combination lets you control exactly what "happens" when your code runs without actually interacting with the real file system.

In [6]:
# Use patching to call `roll_dice` and make sure it returns 6

def roll_dice():
    return random.randint(1, 6)

# Solution
with patch('random.randint', return_value=42):
    result = roll_dice()

In [ ]:
# Call the function `count_letters` to count the number of letters in the word "hello".
# Then use patching to replace `len` with `vowel_len` to only count the vowels.
# TIPP: the `len` function must be addressed as `'builtin.len`.

def count_letters(text):
    return len(text)

def vowel_len(text):
    return sum(1 for char in text if char.lower() in 'aeiou')

# Solution
with patch('builtins.len', vowel_len):
    print(count_letters("hello"))  # Shows only vowel count

2


In [ ]:
# Define a function that does nothing and 

def say_hi_to(name):
    print("Hi " + name)

# Solution:
def do_nothing(text):
    return

with patch('builtins.print', do_nothing):
    say_hi_to("Ada")
    

In [ ]:
# What are the contents and data type of the `.aspect` attribute of the mock window defined below?
# Call the mock window's `.flip()` method. What are the contents and data type of the returned value 

mock_win = Mock(spec=visual.Window)

# Solution
print(type(mock_win.aspect))
x = mock_win.flip()
print(type(x))

In [ ]:
# Use the mock window to draw a Circle. What does the resulting error message tell you about the mock window?

# Solution
mock_win = Mock(spec=visual.Window)
rect = visual.Rect(mock_win)

In [ ]:
# Create a Mock for PsychoPy's `Keyboard` class and use the Mock key defined below as return value for the `getKeys` function.
# Call the `getKeys`

key = Mock(spec=keyboard.KeyPress)
key.name = "left"
key.rt = 0.5

# Solution
mock_kb = Mock(spec=keyboard.Keyboard)
mock_kb.getKeys.return_value = [key]
response = mock_kb.getKeys()
print(f"Reaction time: {response[0].rt}")
print(f"Key pressed: {response[0].name}")

In [ ]:
# Replace the Keyboard with a Mock where the waitKeys method is replaced with the `mock_waitKeys` function defined below
# Then, call `guess_note` to make sure it runs without you having to press a key

kb = keyboard.Keyboard()

def guess_note():
    note = random.choice(["A", "B", "C"])
    tone = sound.Sound(note, secs=0.1, stereo=True)
    kb.clock.reset()
    tone.play()
    keys = kb.waitKeys(keyList=["a", "b", "c"])
    print("The played tone was a " + note)
    print("Your guess was " + keys[0].name.upper())
    print("Your reaction time was " + str(keys[0].rt))

def mock_waitKeys(keyList):

    key = Mock(spec=keyboard.KeyPress)
    key.name = random.choice(keyList)
    key.rt = random.random()
    keys = [key]
    return keys

# Solution
kb = Mock(spec=keyboard.Keyboard)
kb.waitKeys = mock_waitKeys
guess_note()


The played tone was a A
Your guess was B
Your reaction time was 0.14976930001366406


In [ ]:
# Repeat the same task. However, because the Keyboard is now defined within the `guess_note` function,
#  you will have to use patching to replace the `Keyboard` class within the function with the mockwith patch.

def guess_note():
    kb = keyboard.Keyboard()
    note = random.choice(["A", "B", "C"])
    tone = sound.Sound(note, secs=0.1, stereo=True)
    kb.clock.reset()
    tone.play()
    keys = kb.waitKeys(keyList=["a", "b", "c"])
    print("The played tone was a " + note)
    print("Your guess was " + keys[0].name.upper())
    print("Your reaction time was " + str(keys[0].rt))

def mock_waitKeys(keyList):

    key = Mock(spec=keyboard.KeyPress)
    key.name = random.choice(keyList)
    key.rt = random.random()
    keys = [key]
    return keys

# Solution
kb = Mock(spec=keyboard.Keyboard)
kb.waitKeys = mock_waitKeys
with patch('psychopy.hardware.keyboard.Keyboard', return_value=kb):
    guess_note()

The played tone was a B
Your guess was C
Your reaction time was 0.5764683617919284


## 2. Fixtures

In [ ]:
%%ipytest -s
# Modify the code blow so the `test_something` function uses the `announce` fixture.
# Then, run the cell and observe the output.
# Remove the yield keyword --- how does that change the output and what does that tell you about how yield works?

@pytest.fixture
def announce():
    print("Starting...")
    yield 42
    print("Cleanup...")

def test_something():
    print("Working...")


# Solution
def test_something(announce):
    print("Working...")


In [ ]:
%%ipytest -s
# Modify the `test_something` function so that it uses the `announce` fixture and prints out the value yielded by announce

@pytest.fixture
def announce():
    print("Starting...")
    yield 42
    print("Cleanup...")

def test_something():
    print("Working...")

# Solution

def test_something(announce):
    print(announce)
    print("Working...")
 


In [ ]:
%%ipytest -s
# Modify the code blow so the `test_something` function uses the `announce1`  and `announce2` fixtures.
# what determines the order in which the "Starting" and "Cleanup" messages of the fixtures
# TIPP: When patching, the Keyboard class, you must use its full import Path `psychopy.hardware.keyboard.Keyboard`

@pytest.fixture
def announce1():
    print("(1) Starting...")
    yield
    print("(1) Cleanup...")

@pytest.fixture
def announce2():
    print("(2) Starting...")
    yield
    print("(2) Cleanup...")


def test_something():
    print("Working...")


# Solution
def test_something(announce1, announce2):
    print("Working...")
    print(announce)


In [ ]:
%%ipytest
# Write a test function called `test_waitKeys` that uses the `mock_waitKeys` fixture.
# This test function should wait for a press of the left or right arrow key and assert that 
# The name of the returned key is either "left" or "right" and that the reaction time is below 2 seconds

kb = keyboard.Keyboard()

def randomKey(self, keyList):
    key = Mock(spec=keyboard.KeyPress)
    key.rt = random.random()
    key.name = random.choice(keyList)
    return [key]

@pytest.fixture 
def mock_waitKeys():
    with patch('psychopy.hardware.keyboard.Keyboard.waitKeys', randomKey):
        yield

# Solution
def test_waitKeys(mock_waitKeys):
    key = kb.waitKeys(keyList=["left", "right"])
    assert key[0].rt < 2
    assert key[0].name in ["left", "right"]
    

.
1 passed in 0.01s


In [ ]:
%%ipytest
# Write a fixture that uses `patch.object` to replace the window with a mock so that the test runs without ever creating an actual window.

def test_window():
    win = visual.Window()
    win.flip()
    win.close()

# Solution
@pytest.fixture
def mock_window():
    win = Mock(spec=visual.Window)
    with patch("psychopy.visual.Window", return_val=win):
        yield
    
def test_window(mock_window):
    win = visual.Window()
    win.flip()
    win.close()

.                                                                                            [100%]
1 passed in 0.02s


In [ ]:
%%ipytest

# The coce below contains a function that runs a little "experiment" where (after a certain delay) a circle shows up and the
# participant has react by pressing space as quickly as possible. The function returns the duration from ithe start of the trial
# to the time where the key was pressed. Below the function, there are three fixtures to mock the Keyboard, the Window and the
# Cirlce that is drawn.
# Write a test function for `detect_circle` that uses all three fixtures and asserts that the returned trial duration is correct.

def detect_circle(delay):
    kb = keyboard.Keyboard()
    win = visual.Window()
    t_start = core.getTime()
    core.wait(delay)
    circle = visual.Circle(win, size=0.01, pos=(0,0), fillColor="white")
    circle.draw()
    win.flip()
    kb.waitKeys(keyList = ['space'])
    t_stop = core.getTime()
    win.close()
    return t_stop - t_start


@pytest.fixture
def mock_Circle():
    circle = Mock(spec=visual.Circle)
    with patch('psychopy.visual.Circle', return_val=circle):
        yield

@pytest.fixture
def mock_Window():
    win = Mock(spec=visual.Window)
    with patch('psychopy.visual.Window', return_val=win):
        yield

@pytest.fixture
def mock_Keyboard():
    kb = Mock(spec=keyboard.Keyboard)
    with patch('psychopy.hardware.keyboard.Keyboard', return_val=kb):
        yield

# Solution
def test_detect_circle(mock_Circle, mock_Window, mock_Keyboard):
    delay = 0.6
    rt = detect_circle(delay)
    assert abs(rt-delay) < 0.015

.                                                                                            [100%]
1 passed in 0.63s


## 3. Analyzing Simulated Data

In [ ]:
# Run a full experiment while mocking the keyboard.
# Use this to "simulate" several participants and analyze their data to make sure our statistical analysis does not reject the null hypothesis
# The purpose of this is to show that you can translate the logic of testing to data analysis as well